<a href="https://colab.research.google.com/github/caimiao0714/optimization_stats_case_study/blob/master/02Statistical_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating risk data

*Author: Miao Cai* [miao.cai@slu.edu](miao.cai@slu.edu)

## Statistical modeling

We then use four different models to model the risk during the trip:

- Logistic regression
- Poisson regression
- Neural networks

In [4]:
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import sys

print("Python version: " + sys.version)
print("pandas version: " + pd.__version__)
print("tensorflow version: " + tf.__version__)
print("keras version: " + keras.__version__)

Python version: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
pandas version: 0.25.3
tensorflow version: 1.15.0
keras version: 2.2.5
